In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
import pandas as pd
import re
import time

#tfe.enable_eager_execution()

In [52]:
#기본 값
df_path = './dataset'
buckets = [(12, 40)]
PAD_ID = 0
GO_ID = 1
EOS_ID = 2
UNK_ID = 3
user_vocab_size = 9687
bot_vocab_size = 12169

# 1. Load Dataset

Data from cornell dialogue: https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html

In [53]:
# Load the data
lines = open(df_path + '/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conv_lines = open(df_path + '/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [54]:
# Create a dictionary to map each line's id with its text
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]
        
# Create a list of all of the conversations' lines' ids.
convs = [ ]
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))
    
# Sort the sentences into questions (inputs) and answers (targets)
questions = []
answers = []

for conv in convs:
    for i in range(len(conv)-1):
        questions.append(id2line[conv[i]])
        answers.append(id2line[conv[i+1]])

In [55]:
# Check if we have loaded the data correctly
limit = 0
for i in range(limit, limit+5):
    print(questions[i])
    print(answers[i])
    print()

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.

Well, I thought we'd start with pronunciation, if that's okay with you.
Not the hacking and gagging and spitting part.  Please.

Not the hacking and gagging and spitting part.  Please.
Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?

You're asking me out.  That's so cute. What's your name again?
Forget it.

No, no, it's my fault -- we didn't have a proper introduction ---
Cameron.



In [56]:
# Compare lengths of questions and answers
print(len(questions))
print(len(answers))

221616
221616


In [57]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
    
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [58]:
# Take a look at some of the data to ensure that it has been cleaned well.
limit = 0
for i in range(limit, limit+5):
    print(clean_questions[i])
    print(clean_answers[i])
    print()

can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again
well i thought we would start with pronunciation if that is okay with you

well i thought we would start with pronunciation if that is okay with you
not the hacking and gagging and spitting part  please

not the hacking and gagging and spitting part  please
okay then how about we try out some french cuisine  saturday  night

you are asking me out  that is so cute that is your name again
forget it

no no it is my fault  we did not have a proper introduction 
cameron



In [59]:
# Find the length of sentences
lengths = []
for question in clean_questions:
    lengths.append(len(question.split()))
for answer in clean_answers:
    lengths.append(len(answer.split()))

# Create a dataframe so that the values can be inspected
lengths = pd.DataFrame(lengths, columns=['counts'])

In [60]:
lengths.describe()

,counts
count,443232.000000
mean,10.872094
std,12.215895
min,0.000000
25%,4.000000
50%,7.000000
75%,14.000000
max,555.000000


In [61]:
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))
print(np.percentile(lengths, 99))

16.0
19.0
24.0
32.0
58.0


In [62]:
# Remove questions and answers that are shorter than 2 words and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

i = 0
for question in clean_questions:
    if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
        short_questions_temp.append(question)
        short_answers_temp.append(clean_answers[i])
    i += 1

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

i = 0
for answer in short_answers_temp:
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
    i += 1

In [63]:
# Compare the number of lines we will use with the total number of lines.
print("# of questions:", len(short_questions))
print("# of answers:", len(short_answers))
print("% of data used: {}%".format(round(len(short_questions)/len(questions),4)*100))

# of questions: 138335
# of answers: 138335
% of data used: 62.419999999999995%


In [64]:
# Create a dictionary for the frequency of the vocabulary
vocab = {}
for question in short_questions:
    for word in question.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1
            
for answer in short_answers:
    for word in answer.split():
        if word not in vocab:
            vocab[word] = 1
        else:
            vocab[word] += 1

In [65]:
# Remove rare words from the vocabulary.
# We will aim to replace fewer than 5% of words with <UNK>
# You will see this ratio soon.
threshold = 10
count = 0
for k,v in vocab.items():
    if v >= threshold:
        count += 1
        
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 45618
Size of vocab we will use: 8092


In [66]:
# In case we want to use a different vocabulary sizes for the source and target text, 
# we can set different threshold values.
# Nonetheless, we will create dictionaries to provide a unique integer for each word.
questions_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        questions_vocab_to_int[word] = word_num
        word_num += 1
        
answers_vocab_to_int = {}

word_num = 0
for word, count in vocab.items():
    if count >= threshold:
        answers_vocab_to_int[word] = word_num
        word_num += 1

In [67]:
# Add the unique tokens to the vocabulary dictionaries.
codes = ['<PAD>','<EOS>','<UNK>','<GO>']

for code in codes:
    questions_vocab_to_int[code] = len(questions_vocab_to_int)+1
    
for code in codes:
    answers_vocab_to_int[code] = len(answers_vocab_to_int)+1
    
# Create dictionaries to map the unique integers to their respective words.
# i.e. an inverse dictionary for vocab_to_int.
questions_int_to_vocab = {v_i: v for v, v_i in questions_vocab_to_int.items()}
answers_int_to_vocab = {v_i: v for v, v_i in answers_vocab_to_int.items()}

In [68]:
# Check the length of the dictionaries.
print(len(questions_vocab_to_int))
print(len(questions_int_to_vocab))
print(len(answers_vocab_to_int))
print(len(answers_int_to_vocab))

8096
8096
8096
8096


In [69]:
# Add the end of sentence token to the end of every answer.
for i in range(len(short_answers)):
    short_answers[i] += ' <EOS>'

In [70]:
# Convert the text to integers. 
# Replace any words that are not in the respective vocabulary with <UNK> 
questions_int = []
for question in short_questions:
    ints = []
    for word in question.split():
        if word not in questions_vocab_to_int:
            ints.append(questions_vocab_to_int['<UNK>'])
        else:
            ints.append(questions_vocab_to_int[word])
    questions_int.append(ints)
    
answers_int = []
for answer in short_answers:
    ints = []
    for word in answer.split():
        if word not in answers_vocab_to_int:
            ints.append(answers_vocab_to_int['<UNK>'])
        else:
            ints.append(answers_vocab_to_int[word])
    answers_int.append(ints)

In [71]:
# Check the lengths
print(len(questions_int))
print(len(answers_int))

138335
138335


In [72]:
# Calculate what percentage of all words have been replaced with <UNK>
word_count = 0
unk_count = 0

for question in questions_int:
    for word in question:
        if word == questions_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
for answer in answers_int:
    for word in answer:
        if word == answers_vocab_to_int["<UNK>"]:
            unk_count += 1
        word_count += 1
    
unk_ratio = round(unk_count/word_count,4)*100
    
print("Total number of words:", word_count)
print("Number of times <UNK> is used:", unk_count)
print("Percent of words that are <UNK>: {}%".format(round(unk_ratio,3)))

Total number of words: 2334533
Number of times <UNK> is used: 92436
Percent of words that are <UNK>: 3.96%


In [73]:
# Sort questions and answers by the length of questions.
# This will reduce the amount of padding during training
# Which should speed up training and help to reduce the loss

sorted_questions = []
sorted_answers = []

for length in range(1, max_line_length+1):
    for i in enumerate(questions_int):
        if len(i[1]) == length:
            sorted_questions.append(questions_int[i[0]])
            sorted_answers.append(answers_int[i[0]])

print(len(sorted_questions))
print(len(sorted_answers))
print()
for i in range(3):
    print(sorted_questions[i])
    print(sorted_answers[i])
    print()

138335
138335

[58, 48]
[1, 59, 59, 59, 60, 61, 62, 1, 63, 12, 64, 36, 65, 66, 8094]

[1, 84]
[11, 192, 174, 55, 61, 21, 6, 8095, 159, 11, 8094]

[0, 85]
[153, 8, 9, 165, 11, 467, 55, 271, 8094]



In [2]:
#데이터 전처리

#https://github.com/zsdonghao/seq2seq-chatbot/blob/master/main_simple_seq2seq.py
import tensorlayer as tl
from tensorlayer.layers import *

from data.twitter import data
metadata, idx_q, idx_a = data.load_data(PATH='data/twitter/')                   # Twitter

In [3]:
(trainX, trainY), (testX, testY), (validX, validY) = data.split_dataset(idx_q, idx_a)

trainX = trainX.tolist()
trainY = trainY.tolist()
testX = testX.tolist()
testY = testY.tolist()
validX = validX.tolist()
validY = validY.tolist()

In [4]:
trainX = tl.prepro.remove_pad_sequences(trainX)
trainY = tl.prepro.remove_pad_sequences(trainY)
testX = tl.prepro.remove_pad_sequences(testX)
testY = tl.prepro.remove_pad_sequences(testY)
validX = tl.prepro.remove_pad_sequences(validX)
validY = tl.prepro.remove_pad_sequences(validY)

In [5]:
###============= parameters
xseq_len = len(trainX)#.shape[-1]
yseq_len = len(trainY)#.shape[-1]
assert xseq_len == yseq_len
batch_size = 32
n_step = int(xseq_len/batch_size)
xvocab_size = len(metadata['idx2w']) # 8002 (0~8001)
emb_dim = 1024

w2idx = metadata['w2idx']   # dict  word 2 index
idx2w = metadata['idx2w']   # list index 2 word
unk_id = w2idx['unk']   # 1
pad_id = w2idx['_']     # 0

start_id = xvocab_size  # 8002
end_id = xvocab_size+1  # 8003

In [6]:
w2idx.update({'start_id': start_id})
w2idx.update({'end_id': end_id})
idx2w = idx2w + ['start_id', 'end_id']

xvocab_size = yvocab_size = xvocab_size + 2

In [7]:
""" A data for Seq2Seq should look like this:
input_seqs : ['how', 'are', 'you', '<PAD_ID'>]
decode_seqs : ['<START_ID>', 'I', 'am', 'fine', '<PAD_ID'>]
target_seqs : ['I', 'am', 'fine', '<END_ID>', '<PAD_ID'>]
target_mask : [1, 1, 1, 1, 0]
"""

print("encode_seqs", [idx2w[id] for id in trainX[10]])
target_seqs = tl.prepro.sequences_add_end_id([trainY[10]], end_id=end_id)[0]
    # target_seqs = tl.prepro.remove_pad_sequences([target_seqs], pad_id=pad_id)[0]
print("target_seqs", [idx2w[id] for id in target_seqs])
decode_seqs = tl.prepro.sequences_add_start_id([trainY[10]], start_id=start_id, remove_last=False)[0]
    # decode_seqs = tl.prepro.remove_pad_sequences([decode_seqs], pad_id=pad_id)[0]
print("decode_seqs", [idx2w[id] for id in decode_seqs])
target_mask = tl.prepro.sequences_get_mask([target_seqs])[0]
print("target_mask", target_mask)
print(len(target_seqs), len(decode_seqs), len(target_mask))


encode_seqs ['wednesday', 'and', 'thursday', 'in', 'seattle', 's', 'big', 'conference', '']
target_seqs ['wish', 'but', 'cant', 'travel', 'enjoy', 'end_id']
decode_seqs ['start_id', 'wish', 'but', 'cant', 'travel', 'enjoy']
target_mask [1 1 1 1 1 1]
6 6 6


In [8]:
###============= model
def model(encode_seqs, decode_seqs, is_train=True, reuse=False):
    with tf.variable_scope("model", reuse=reuse):
        # for chatbot, you can use the same embedding layer,
        # for translation, you may want to use 2 seperated embedding layers
        with tf.variable_scope("embedding") as vs:
            net_encode = EmbeddingInputlayer(
                inputs = encode_seqs,
                vocabulary_size = xvocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
            vs.reuse_variables()
            tl.layers.set_name_reuse(True)
            net_decode = EmbeddingInputlayer(
                inputs = decode_seqs,
                vocabulary_size = xvocab_size,
                embedding_size = emb_dim,
                name = 'seq_embedding')
        net_rnn = Seq2Seq(net_encode, net_decode,
                cell_fn = tf.contrib.rnn.BasicLSTMCell,
                n_hidden = emb_dim,
                initializer = tf.random_uniform_initializer(-0.1, 0.1),
                encode_sequence_length = retrieve_seq_length_op2(encode_seqs),
                decode_sequence_length = retrieve_seq_length_op2(decode_seqs),
                initial_state_encode = None,
                dropout = (0.5 if is_train else None),
                n_layer = 3,
                return_seq_2d = True,
                name = 'seq2seq')
        net_out = DenseLayer(net_rnn, n_units=xvocab_size, act=tf.identity, name='output')
    return net_out, net_rnn

# model for training
encode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="encode_seqs")
decode_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="decode_seqs")
target_seqs = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_seqs")
target_mask = tf.placeholder(dtype=tf.int64, shape=[batch_size, None], name="target_mask") # tl.prepro.sequences_get_mask()
net_out, _ = model(encode_seqs, decode_seqs, is_train=True, reuse=False)

# model for inferencing
encode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="encode_seqs")
decode_seqs2 = tf.placeholder(dtype=tf.int64, shape=[1, None], name="decode_seqs")
net, net_rnn = model(encode_seqs2, decode_seqs2, is_train=False, reuse=True)
y = tf.nn.softmax(net.outputs)

# loss for training
    # print(net_out.outputs)    # (?, 8004)
    # print(target_seqs)    # (32, ?)
    # loss_weights = tf.ones_like(target_seqs, dtype=tf.float32)
    # loss = tf.contrib.legacy_seq2seq.sequence_loss(net_out.outputs, target_seqs, loss_weights, yvocab_size)
loss = tl.cost.cross_entropy_seq_with_mask(logits=net_out.outputs, target_seqs=target_seqs, input_mask=target_mask, return_details=False, name='cost')

net_out.print_params(False)

lr = 0.0001
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
# Truncated Backpropagation for training (option)
# max_grad_norm = 30
# grads, _ = tf.clip_by_global_norm(tf.gradients(loss, net_out.all_params),max_grad_norm)
# optimizer = tf.train.GradientDescentOptimizer(lr)
# train_op = optimizer.apply_gradients(zip(grads, net_out.all_params))

# sess = tf.InteractiveSession()
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
tl.files.load_and_assign_npz(sess=sess, name='n.npz', network=net)

###============= train
n_epoch = 50
for epoch in range(n_epoch):
    epoch_time = time.time()
    ## shuffle training data
    from sklearn.utils import shuffle
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    ## train an epoch
    total_err, n_iter = 0, 0
    for X, Y in tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False):
        step_time = time.time()

        X = tl.prepro.pad_sequences(X)
        _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
        _target_seqs = tl.prepro.pad_sequences(_target_seqs)

        _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
        _decode_seqs = tl.prepro.pad_sequences(_decode_seqs)
        _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

        ## you can view the data here
        # for i in range(len(X)):
        #     print(i, [idx2w[id] for id in X[i]])
        #     # print(i, [idx2w[id] for id in Y[i]])
        #     print(i, [idx2w[id] for id in _target_seqs[i]])
        #     print(i, [idx2w[id] for id in _decode_seqs[i]])
        #     print(i, _target_mask[i])
        #     print(len(_target_seqs[i]), len(_decode_seqs[i]), len(_target_mask[i]))
        # exit()

        _, err = sess.run([train_op, loss],
                        {encode_seqs: X,
                        decode_seqs: _decode_seqs,
                        target_seqs: _target_seqs,
                        target_mask: _target_mask})

        if n_iter % 200 == 0:
            print("Epoch[%d/%d] step:[%d/%d] loss:%f took:%.5fs" % (epoch, n_epoch, n_iter, n_step, err, time.time() - step_time))

        total_err += err; n_iter += 1

        ###============= inference
        if n_iter % 1000 == 0:
            seeds = ["happy birthday have a nice day",
                    "donald trump won last nights presidential debate according to snap online polls"]
            for seed in seeds:
                print("Query >", seed)
                seed_id = [w2idx[w] for w in seed.split(" ")]
                for _ in range(5):  # 1 Query --> 5 Reply
                    # 1. encode, get state
                    state = sess.run(net_rnn.final_state_encode,
                                    {encode_seqs2: [seed_id]})
                    # 2. decode, feed start_id, get first word
                    #   ref https://github.com/zsdonghao/tensorlayer/blob/master/example/tutorial_ptb_lstm_state_is_tuple.py
                    o, state = sess.run([y, net_rnn.final_state_decode],
                                    {net_rnn.initial_state_decode: state,
                                    decode_seqs2: [[start_id]]})
                    w_id = tl.nlp.sample_top(o[0], top_k=3)
                    w = idx2w[w_id]
                    # 3. decode, feed state iteratively
                    sentence = [w]
                    for _ in range(30): # max sentence length
                        o, state = sess.run([y, net_rnn.final_state_decode],
                                        {net_rnn.initial_state_decode: state,
                                        decode_seqs2: [[w_id]]})
                        w_id = tl.nlp.sample_top(o[0], top_k=2)
                        w = idx2w[w_id]
                        if w_id == end_id:
                            break
                        sentence = sentence + [w]
                    print(" >", ' '.join(sentence))

    print("Epoch[%d/%d] averaged loss:%f took:%.5fs" % (epoch, n_epoch, total_err/n_iter, time.time()-epoch_time))

    tl.files.save_npz(net.all_params, name='n.npz', sess=sess)

  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
  [**] Seq2Seq model/seq2seq: n_hidden:1024 cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_encode: n_hidden:1024, in_dim:3 in_shape:(32, ?, 1024) cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
       batch_size (concurrent processes): 32
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_decode: n_hidden:1024, in_dim:3 in_shape:(32, ?, 1024) cell_fn:BasicLSTMCell dropout:0.5 n_layer:3
       batch_size (concurrent processes): 32
  [TL] DenseLayer  model/output: 8004 identity
  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
  [TL] EmbeddingInputlayer model/embedding/seq_embedding: (8004, 1024)
  [**] Seq2Seq model/seq2seq: n_hidden:1024 cell_fn:BasicLSTMCell dropout:None n_layer:3
  [TL] DynamicRNNLayer model/seq2seq/seq2seq_encode: n_hidden:1024, in_dim:3 in_shape:(1, ?, 1024) cell_fn:BasicLS

KeyboardInterrupt: 

# Modeling

https://towardsdatascience.com/text-summarization-with-amazon-reviews-41801c2210b

In [49]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_proba

def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer
    tf.varaible_scope을 사용하여, 변수들을 재 사용 해야함. 잘 모르겠으면 https://www.tensorflow.org/tutorials/word2vec 참조
    bidirectional임으로 마지막에 concat해야함
    '''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            lstm_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                             initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.DropoutWrapper(lstm_fw, input_keep_prob = keep_prob)
            
            lstm_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                             initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.DropoutWrapper(lstm_bw, input_keep_prob = keep_prob)
            
            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw,
                                                                   cell_bw,
                                                                   inputs = rnn_inputs,
                                                                   sequence_length = sequence_length,
                                                                   dtype=tf.float32)
        
        return enc_output, enc_state

In [50]:
#Training Helpler: read a seq of integers from encoding layer
#BasicDecoder: process sequence with decoding cell, and output layer, (fully connected)
#initial_state comes from Dynamic AttentionWrapperState
#dynamic_decode: 학습에 필요한 출력을

def training_decoding_layer(dec_embed_input, summary_length, dec_cell,
                           initial_state, output_layer, vocab_size,
                           max_summary_length):
    
    #encoding layer에 있는 integer sequence를 읽음
    training_helper = tf.contrib.seq2seq.TrainingHelper( 
                         inputs=dec_embed_input,
                         sequence_length=summary_length,
                         time_major=False)
    
    #decoding cell과 output layer를 fully connected layer로 변경
    training_decoder = tf.contrib.seq2seq.BasicDecoder(
                          dec_cell,
                          training_helper,
                          initial_state,
                          output_layer)
    
    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(
                            training_decoder,
                            output_time_major=False,
                            impute_finished=True,
                            maximum_iterations=max_summary_length)
    return training_logits
    
    
def inference_decoding_layer(embeddings, start_token, end_token,        
                             dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):

    
    start_tokens = tf.tile(tf.constant([start_token],  
                                       dtype=tf.int32),  
                                       [batch_size], 
                                       name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(  
                           embeddings,
                           start_tokens,
                           end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(
                            dec_cell,
                            inference_helper,
                            initial_state,
                            output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(
                              inference_decoder,
                              output_time_major=False,
                              impute_finished=True,
                              maximum_iterations=max_summary_length)
    
    return inference_logit

def decoding_layer(dec_embed_input, embeddings, enc_output,  
                   enc_state, vocab_size, text_length, 
                   summary_length, max_summary_length, rnn_size, 
                   vocab_to_int, keep_prob, batch_size, num_layers):

    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                  initializer=tf.random_uniform_initializer(-0.1, 
                                                            0.1,     
                                                            seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(
                           lstm, 
                           input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
           kernel_initializer = tf.truncated_normal_initializer(  
                                    mean=0.0, 
                                    stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(
                      rnn_size,
                      enc_output,
                      text_length,
                      normalize=False,
                      name='BahdanauAttention')
    
        dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(
                        enc_state[0],
                        _zero_state_tensors(rnn_size, 
                                            batch_size, 
                                            tf.float32)) 
    
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(
                              dec_embed_input, 
                              summary_length, 
                              dec_cell, 
                              initial_state,
                              output_layer,
                              vocab_size, 
                              max_summary_length)
    
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(
                              embeddings,  
                              vocab_to_int['<GO>'], 
                              vocab_to_int['<EOS>'],
                              dec_cell, 
                              initial_state, 
                              output_layer,
                              max_summary_length,
                              batch_size)
    return training_logits, inference_logits

NameError: name 'd' is not defined

In [ ]:
def seq2seq_model(input_data, target_data, keep_prob, text_length,  
                  summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int,  
                  batch_size):
    
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, 
                                           text_length, 
                                           num_layers,  
                                           enc_embed_input,  
                                           keep_prob)
    
    dec_input = process_encoding_input(target_data, 
                                       vocab_to_int, 
                                       batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(
         dec_embed_input, 
         embeddings,
         enc_output,
         enc_state, 
         vocab_size, 
         text_length, 
         summary_length, 
         max_summary_length,
         rnn_size, 
         vocab_to_int, 
         keep_prob, 
         batch_size,
         num_layers)
    
    return training_logits, inference_logits

def get_batches(summaries, texts, batch_size):
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(  
                                  summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield (pad_summaries_batch, 
               pad_texts_batch, 
               pad_summaries_lengths, 
               pad_texts_lengths)


In [24]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    return input_data, targets, lr, keep_prob

def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    '''Create the encoding layer'''
    #lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    gru = tf.contrib.rnn.GRUCell(rnn_size)
    drop = tf.contrib.rnn.DropoutWrapper(gru, input_keep_prob = keep_prob)
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = enc_cell,
                                                   cell_bw = enc_cell,
                                                   sequence_length = sequence_length,
                                                   inputs = rnn_inputs, 
                                                   dtype=tf.float32)
    return enc_state

In [26]:
def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    '''Decode the training data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
            tf.contrib.seq2seq.prepare_attention(attention_states,
                                                 attention_option="bahdanau",
                                                 num_units=dec_cell.output_size)
    
    train_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                     att_keys,
                                                                     att_vals,
                                                                     att_score_fn,
                                                                     att_construct_fn,
                                                                     name = "attn_dec_train")
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                              train_decoder_fn, 
                                                              dec_embed_input, 
                                                              sequence_length, 
                                                              scope=decoding_scope)
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    return output_fn(train_pred_drop)

def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id,
                         maximum_length, vocab_size, decoding_scope, output_fn, keep_prob, batch_size):
    '''Decode the prediction data'''
    
    attention_states = tf.zeros([batch_size, 1, dec_cell.output_size])
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(
                                                    )
    
# 버젼 업으로 사라짐
#    att_keys, att_vals, att_score_fn, att_construct_fn = \
#             tf.contrib.seq2seq.prepare_attention(attention_states,
#                                                  attention_option="bahdanau",
#                                                  num_units=dec_cell.output_size)
    
    infer_decoder_fn = tf.contrib.seq2seq.attention_decoder_fn_inference(output_fn, 
                                                                         encoder_state[0], 
                                                                         att_keys, 
                                                                         att_vals, 
                                                                         att_score_fn, 
                                                                         att_construct_fn, 
                                                                         dec_embeddings,
                                                                         start_of_sequence_id, 
                                                                         end_of_sequence_id, 
                                                                         maximum_length, 
                                                                         vocab_size, 
                                                                         name = "attn_dec_inf")
    infer_logits, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell, 
                                                                infer_decoder_fn, 
                                                                scope=decoding_scope)
    
    return infer_logits

def decoding_layer(dec_embed_input, dec_embeddings, encoder_state, vocab_size, sequence_length, rnn_size,
                   num_layers, vocab_to_int, keep_prob, batch_size):
    '''Create the decoding cell and input the parameters for the training and inference decoding layers'''
    
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        dec_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
        
        weights = tf.truncated_normal_initializer(stddev=0.1)
        biases = tf.zeros_initializer()
        output_fn = lambda x: tf.contrib.layers.fully_connected(x, 
                                                                vocab_size, 
                                                                None, 
                                                                scope=decoding_scope,
                                                                weights_initializer = weights,
                                                                biases_initializer = biases)

        train_logits = decoding_layer_train(encoder_state, 
                                            dec_cell, 
                                            dec_embed_input, 
                                            sequence_length, 
                                            decoding_scope, 
                                            output_fn, 
                                            keep_prob, 
                                            batch_size)
        decoding_scope.reuse_variables()
        infer_logits = decoding_layer_infer(encoder_state, 
                                            dec_cell, 
                                            dec_embeddings, 
                                            vocab_to_int['<GO>'],
                                            vocab_to_int['<EOS>'], 
                                            sequence_length - 1, 
                                            vocab_size,
                                            decoding_scope, 
                                            output_fn, keep_prob, 
                                            batch_size)

    return train_logits, infer_logits

In [27]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size, sequence_length, answers_vocab_size, 
                  questions_vocab_size, enc_embedding_size, dec_embedding_size, rnn_size, num_layers, 
                  questions_vocab_to_int):
    
    '''Use the previous functions to create the training and inference logits'''
    
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, 
                                                       answers_vocab_size+1, 
                                                       enc_embedding_size,
                                                       initializer = tf.random_uniform_initializer(0,1))
    enc_state = encoding_layer(enc_embed_input, rnn_size, num_layers, keep_prob, sequence_length)

    dec_input = process_encoding_input(target_data, questions_vocab_to_int, batch_size)
    dec_embeddings = tf.Variable(tf.random_uniform([questions_vocab_size+1, dec_embedding_size], 0, 1))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    train_logits, infer_logits = decoding_layer(dec_embed_input, 
                                                dec_embeddings, 
                                                enc_state, 
                                                questions_vocab_size, 
                                                sequence_length, 
                                                rnn_size, 
                                                num_layers, 
                                                questions_vocab_to_int, 
                                                keep_prob, 
                                                batch_size)
    return train_logits, infer_logits

In [28]:
# Set the Hyperparameters
epochs = 100
batch_size = 128
rnn_size = 512
num_layers = 2
encoding_embedding_size = 512
decoding_embedding_size = 512
learning_rate = 0.005
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.75

In [29]:
# Reset the graph to ensure that it is ready for training
tf.reset_default_graph()
# Start the session
sess = tf.InteractiveSession()
    
# Load the model inputs    
input_data, targets, lr, keep_prob = model_inputs()
# Sequence length will be the max line length for each batch
sequence_length = tf.placeholder_with_default(max_line_length, None, name='sequence_length')
# Find the shape of the input data for sequence_loss
input_shape = tf.shape(input_data)

# Create the training and inference logits
train_logits, inference_logits = seq2seq_model(
    tf.reverse(input_data, [-1]), targets, keep_prob, batch_size, sequence_length, len(answers_vocab_to_int), 
    len(questions_vocab_to_int), encoding_embedding_size, decoding_embedding_size, rnn_size, num_layers, 
    questions_vocab_to_int)

# Create a tensor for the inference logits, needed if loading a checkpoint version of the model
tf.identity(inference_logits, 'logits')

with tf.name_scope("optimization"):
    # Loss function
    cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,
        tf.ones([input_shape[0], sequence_length]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

AttributeError: module 'tensorflow.contrib.seq2seq' has no attribute 'prepare_attention'

In [ ]:
def pad_sentence_batch(sentence_batch, vocab_to_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
def batch_data(questions, answers, batch_size):
    """Batch questions and answers together"""
    for batch_i in range(0, len(questions)//batch_size):
        start_i = batch_i * batch_size
        questions_batch = questions[start_i:start_i + batch_size]
        answers_batch = answers[start_i:start_i + batch_size]
        pad_questions_batch = np.array(pad_sentence_batch(questions_batch, questions_vocab_to_int))
        pad_answers_batch = np.array(pad_sentence_batch(answers_batch, answers_vocab_to_int))
        yield pad_questions_batch, pad_answers_batch

In [ ]:
# Validate the training with 10% of the data
train_valid_split = int(len(sorted_questions)*0.15)

# Split the questions and answers into training and validating data
train_questions = sorted_questions[train_valid_split:]
train_answers = sorted_answers[train_valid_split:]

valid_questions = sorted_questions[:train_valid_split]
valid_answers = sorted_answers[:train_valid_split]

print(len(train_questions))
print(len(valid_questions))

In [ ]:
display_step = 100 # Check training loss after every 100 batches
stop_early = 0 
stop = 5 # If the validation loss does decrease in 5 consecutive checks, stop training
validation_check = ((len(train_questions))//batch_size//2)-1 # Modulus for checking validation loss
total_train_loss = 0 # Record the training loss for each display step
summary_valid_loss = [] # Record the validation loss for saving improvements in the model

checkpoint = "best_model.ckpt" 

sess.run(tf.global_variables_initializer())

for epoch_i in range(1, epochs+1):
    for batch_i, (questions_batch, answers_batch) in enumerate(
            batch_data(train_questions, train_answers, batch_size)):
        start_time = time.time()
        _, loss = sess.run(
            [train_op, cost],
            {input_data: questions_batch,
             targets: answers_batch,
             lr: learning_rate,
             sequence_length: answers_batch.shape[1],
             keep_prob: keep_probability})

        total_train_loss += loss
        end_time = time.time()
        batch_time = end_time - start_time

        if batch_i % display_step == 0:
            print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                  .format(epoch_i,
                          epochs, 
                          batch_i, 
                          len(train_questions) // batch_size, 
                          total_train_loss / display_step, 
                          batch_time*display_step))
            total_train_loss = 0

        if batch_i % validation_check == 0 and batch_i > 0:
            total_valid_loss = 0
            start_time = time.time()
            for batch_ii, (questions_batch, answers_batch) in \
                    enumerate(batch_data(valid_questions, valid_answers, batch_size)):
                valid_loss = sess.run(
                cost, {input_data: questions_batch,
                       targets: answers_batch,
                       lr: learning_rate,
                       sequence_length: answers_batch.shape[1],
                       keep_prob: 1})
                total_valid_loss += valid_loss
            end_time = time.time()
            batch_time = end_time - start_time
            avg_valid_loss = total_valid_loss / (len(valid_questions) / batch_size)
            print('Valid Loss: {:>6.3f}, Seconds: {:>5.2f}'.format(avg_valid_loss, batch_time))
            
            # Reduce learning rate, but not below its minimum value
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate

            summary_valid_loss.append(avg_valid_loss)
            if avg_valid_loss <= min(summary_valid_loss):
                print('New Record!') 
                stop_early = 0
                saver = tf.train.Saver() 
                saver.save(sess, checkpoint)

            else:
                print("No Improvement.")
                stop_early += 1
                if stop_early == stop:
                    break
    
    if stop_early == stop:
        print("Stopping Training.")
        break

In [ ]:
def question_to_seq(question, vocab_to_int):
    '''Prepare the question for the model'''
    
    question = clean_text(question)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in question.split()]

In [ ]:
# Create your own input question
#input_question = 'How are you?'

# Use a question from the data as your input
random = np.random.choice(len(short_questions))
input_question = short_questions[random]

# Prepare the question
input_question = question_to_seq(input_question, questions_vocab_to_int)

# Pad the questions until it equals the max_line_length
input_question = input_question + [questions_vocab_to_int["<PAD>"]] * (max_line_length - len(input_question))
# Add empty questions so the the input_data is the correct shape
batch_shell = np.zeros((batch_size, max_line_length))
# Set the first question to be out input question
batch_shell[0] = input_question    
    
# Run the model with the input question
answer_logits = sess.run(inference_logits, {input_data: batch_shell, 
                                            keep_prob: 1.0})[0]

# Remove the padding from the Question and Answer
pad_q = questions_vocab_to_int["<PAD>"]
pad_a = answers_vocab_to_int["<PAD>"]

print('Question')
print('  Word Ids:      {}'.format([i for i in input_question if i != pad_q]))
print('  Input Words: {}'.format([questions_int_to_vocab[i] for i in input_question if i != pad_q]))

print('\nAnswer')
print('  Word Ids:      {}'.format([i for i in np.argmax(answer_logits, 1) if i != pad_a]))
print('  Response Words: {}'.format([answers_int_to_vocab[i] for i in np.argmax(answer_logits, 1) if i != pad_a]))

## Preprocessing Dataset

- Datset API사용 (큐 기반)
- Clear Structure and Simplicity

In [ ]:
def model_inputs():
    
    input_data = tf.placeholder(tf.int32, 
                                [None, None], 
                                name='input')
    
    targets = tf.placeholder(tf.int32, 
                             [None, None], 
                             name='targets')
    
    lr = tf.placeholder(tf.float32, name='learning_rate')
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return input_data, targets, lr, keep_prob

def process_encoding_input(target_data, vocab_to_int, batch_size):
    
    ending = tf.strided_slice(target_data, 
                              [0, 0], 
                              [batch_size, -1], 
                              [1, 1]) #마지막에 있는 단어 토큰을 제거함
    
    dec_input = tf.concat([tf.fill([batch_size, 1], 
                                   vocab_to_int['<GO>']), 
                           ending], 1)
    return dec_input

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   sequence_length, attn_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
    drop = tf.contrib.rnn.DropoutWrapper(
               lstm, 
               input_keep_prob = keep_prob)
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    _, enc_state = tf.nn.bidirectional_dynamic_rnn(
               cell_fw = enc_cell,
               cell_bw = enc_cell,
               sequence_length = sequence_length,
               inputs = rnn_inputs, 
               dtype=tf.float32)
    return enc_state

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         sequence_length, decoding_scope,
                         output_fn, keep_prob, batch_size):
    
    attention_states = tf.zeros([batch_size, 
                                1, 
                                dec_cell.output_size])
    
    att_keys, att_vals, att_score_fn, att_construct_fn = \
        tf.contrib.seq2seq.prepare_attention(
             attention_states,
             attention_option="bahdanau",
             num_units=dec_cell.output_size)
    
    train_decoder_fn = \  
       tf.contrib.seq2seq.attention_decoder_fn_train(
             encoder_state[0],
             att_keys,
             att_vals,
             att_score_fn,
             att_construct_fn,
             name = "attn_dec_train")
    
    train_pred, _, _ = tf.contrib.seq2seq.dynamic_rnn_decoder(
             dec_cell, 
             train_decoder_fn, 
             dec_embed_input, 
             sequence_length, 
             scope=decoding_scope)
    
    train_pred_drop = tf.nn.dropout(train_pred, keep_prob)
    
    return output_fn(train_pred_drop)

In [ ]:
#Load Sample Dataset

def read_train_set(train_path, max_size=None):
    train_set = [[] for _ in range(buckets[0][0])]
    
    with open(train_path, "r", encoding="utf-8") as ts:
        counter = 0
        while not max_size or counter < max_size:
            
            # iterator 성질을 이용하여, 유저, 봇 발화 불러오기
            user_tokens, bot_tokens = ts.readline(), ts.readline()
            
            # 유저 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            user_token_ids = [int(x) for x in user_tokens.split()]

            # 모르는 토큰이 있으면 언노운 아이디 부여
            for i, user_token_id in enumerate(user_token_ids):
                if user_token_id > user_vocab_size:
                    user_token_ids[i] = UNK_ID

            # 유저 버킷 크기로 잘라낸다
            user_token_bucket = user_token_ids[:buckets[0][0]]
            user_token_len = len(user_token_bucket)

            # 봇 발화에서 모르는 토큰이 있으면 언노운 아이디 부여한다
            bot_token_ids = [int(x) for x in bot_tokens.split()]
            for i, bot_token_id in enumerate(bot_token_ids):
                if bot_token_id > bot_vocab_size:
                    bot_token_ids[i] = UNK_ID

            # 봇 버킷 크기로 잘라낸다
            bot_token_bucket = bot_token_ids[:(buckets[0][1]-1)]

            # 유저 토큰 길이에 따라 유저 발화와 봇 발화를 모은다
            train_set[user_token_len - 1].append([user_token_bucket, bot_token_bucket])

            counter += 1

            # 진행도를 알아볼 수 있도록 1000번마다 프린트한다
            if counter % 1000 == 0:
                print("read data line %d" % counter)

        return train_set
    
train_set = read_train_set(df_path, 5000)
train_len_hist = [len(train_set[i]) for i in range(len(train_set))]
train_size = float(sum(train_len_hist))

print("sep by train, {}".format(train_len_hist))
print("train_size: {}".format(train_size))

In [ ]:
# train_set[4][0][0] #고객 발화
# train_set[4][0][1] #봇 발화

df_train = [train_set[4][i] for i in range(len(train_set[4]))] #Sample로 input size 가 5인 발화를 추출한다.
df_user = [df_train[i][0] for i in range(len(df_train))]
df_bot = [df_train[i][1] for i in range(len(df_train))]

In [ ]:
embed_size = 1
window_size = 3
dict_size = 5
filter_size = window_size*embed_size

ids = tf.constant(df_train)

#https://tutorials.botsfloor.com/how-to-build-your-first-chatbot-c84495d4622d



def input_fn():
    ids = tf.constant([[5, 6, 10, 9, 2], [5, 6, 10, 60, 2]])
    label_ids = tf.constant([[4, 22, 163, 649, 2], [4, 22, 163, 649, 2]])
    return {'ids': ids,
            'init_label': label_ids[:, :window_size],
            'label_length': int(label_ids.shape[1])-window_size
            }, label_ids[:, window_size:]

In [ ]:
ids = tf.constant([[1, 2, 3, 4, 3, 2, 1], [1, 2, 3, 2, 3, 2, 3]])
label_ids = tf.constant([[0, 0, 0, 1, 3, 1, 3, 2], [0, 0, 0, 2, 1, 3, 4, 4]]) #무조건 앞에 3개를... 
init_label = label_ids[:, :window_size]
label_length = int(label_ids.shape[1]) - window_size
label_ids = label_ids[:, window_size:]

print(init_label)
print(label_length)
print(label_ids)

In [ ]:
embeddings = tfe.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)
print(embeddings.shape)

In [ ]:
class Embedding(tf.layers.Layer):
    """ 임베딩 층"""
    def __init__(self, vocab_size, embedding_dim, **kwargs):
        super(Embedding, self).__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        
    def build(self, _):
        self.embedding = self.add_variable(
        "embedding_kernel",
        shape=[self.vocab_size, self.embedding_dim],
        dtype=tf.float32,
        initializer = tf.random_uniform_initializer(-0.1, 0.1),
        trainable=True)
    
    def call(self, x):
        return tf.nn.embedding_lookup(self.embedding, x)
    

Embedding(50, 100)

In [ ]:
# embedding
input_embeds = tf.nn.embedding_lookup(embeddings, ids)
print(input_embeds)

input_flat = tf.layers.flatten(input_embeds)
input_flat = tf.expand_dims(input_flat, -1)

init_label = tf.nn.embedding_lookup(embeddings, init_label)
init_label = tf.layers.flatten(init_label)
init_label = tf.expand_dims(init_label, -1)

print(input_flat.shape, init_label.shape)

label_onehot = tf.one_hot(label_ids, depth=dict_size, dtype=tf.float32)
print(label_onehot)

In [ ]:
# encoder
encoder_conv = tf.layers.conv1d(
        inputs=input_flat,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size,
        padding='same')

print(encoder_conv.shape)
#embed_size, 반으로 나눈다.
encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])
print(encoder_glu.shape)

In [ ]:
#decoder: 그림에서 왼쪽에서 하나의 블록을 구하는 식

decoder_conv = tf.layers.conv1d(
        inputs=init_label,
        filters=2*embed_size,
        kernel_size=filter_size,
        strides=embed_size)

decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

print(decoder_conv.shape)
print(decoder_glu.shape)

tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1]) #decoder 단어 1개, encoder는 7개단어)
print(tiled_decoder_glu.shape)

dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)
print(dot_prod.shape)

In [ ]:
attention = tf.nn.softmax(dot_prod, axis=1) #수정해야함
#tf.reduce_sum(attention, axis=1)
print(attention.shape)

z_plus_e = encoder_glu + input_embeds #why???
print(z_plus_e.shape)

tiled_attention = tf.tile(attention, [1, 1, embed_size])
print(tiled_attention.shape)

c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
print(c.shape)

decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])
print(decoder_glu.shape)

logits = tf.layers.dense(c+decoder_glu, dict_size)
print(logits.shape)

out = tf.nn.softmax(logits)
print(out)

next_id = tf.argmax(out, axis=1)
next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
next_embeds = tf.expand_dims(next_embeds, -1)
decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

next_ids.append(next_id)
outs.append(logits)

In [ ]:
c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

logits = tf.layers.dense(c+decoder_glu, dict_size)

out = tf.nn.softmax(logits) #out으로 묶기

next_id = tf.argmax(out, axis=1)
next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
next_embeds = tf.expand_dims(next_embeds, -1)
decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

next_ids.append(next_id)
outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

if TRAIN:
    global_step = tf.train.get_global_step()
    loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
    train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
    estimator_spec = tf.estimator.EstimatorSpec(
            mode=mode,
            train_op=train_op,
            loss=loss)

In [ ]:
#decoder
next_ids = []
outs = []
for l in range(label_length):
    if l == 0: decoder_input = init_label

    decoder_conv = tf.layers.conv1d(
            inputs=init_label,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size)


    decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

    tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


    dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

    attention = tf.nn.softmax(dot_prod)

    z_plus_e = encoder_glu + input_embeds

    tiled_attention = tf.tile(attention, [1, 1, embed_size])

    c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
    decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

    logits = tf.layers.dense(c+decoder_glu, dict_size)

    out = tf.nn.softmax(logits)

    next_id = tf.argmax(out, axis=1)
    next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
    next_embeds = tf.expand_dims(next_embeds, -1)
    decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1) #옆에 붙이자, 기존꺼 버리고..

    next_ids.append(next_id)
    outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

print(outs)
print(next_ids)

In [ ]:
loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss)

In [ ]:
encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])

#decoder
next_ids = []
outs = []
for l in range(features['label_length']):
    if l == 0: decoder_input = init_label

    decoder_conv = tf.layers.conv1d(
            inputs=init_label,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size)


    decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

    tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


    dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

    attention = tf.nn.softmax(dot_prod)

    z_plus_e = encoder_glu + input_embeds

    tiled_attention = tf.tile(attention, [1, 1, embed_size])

    c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
    decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

    logits = tf.layers.dense(c+decoder_glu, dict_size)

    out = tf.nn.softmax(logits)

    next_id = tf.argmax(out, axis=1)
    next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
    next_embeds = tf.expand_dims(next_embeds, -1)
    decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1) #옆에 붙이자, 기존꺼 버리고..

    next_ids.append(next_id)
    outs.append(logits)

outs = tf.stack(outs, axis=1)
next_ids = tf.stack(next_ids, axis=1)

if TRAIN:
    global_step = tf.train.get_global_step()
    loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
    train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
    estimator_spec = tf.estimator.EstimatorSpec(
            mode=mode,
            train_op=train_op,
            loss=loss)


In [ ]:
def model_fn(mode, features, labels):
    TRAIN = mode == tf.estimator.ModeKeys.TRAIN
    EVAL = mode == tf.estimator.ModeKeys.EVAL
    PREDICT = mode == tf.estimator.ModeKeys.PREDICT

    embeddings = tf.Variable(tf.truncated_normal([dict_size, embed_size]), trainable=False)

    # embedding
    input_embeds = tf.nn.embedding_lookup(embeddings, features['ids'])

    input_flat = tf.layers.flatten(input_embeds)
    input_flat = tf.expand_dims(input_flat, -1)

    init_label = tf.nn.embedding_lookup(embeddings, features['init_label'])
    init_label = tf.layers.flatten(init_label)
    init_label = tf.expand_dims(init_label, -1)

    if not PREDICT:
        label_onehot = tf.one_hot(labels, depth=dict_size, dtype=tf.float32)

    # encoder
    encoder_conv = tf.layers.conv1d(
            inputs=input_flat,
            filters=2*embed_size,
            kernel_size=filter_size,
            strides=embed_size,
            padding='same')

    encoder_glu = encoder_conv[:, :, embed_size:]*tf.nn.sigmoid(encoder_conv[:, :, :embed_size])



        decoder_glu = decoder_conv[:, :, embed_size:]*tf.nn.sigmoid(decoder_conv[:, :, :embed_size])

        tiled_decoder_glu = tf.tile(decoder_glu, [1, int(encoder_glu.shape[1]), 1])


        dot_prod = tf.matmul(encoder_glu, decoder_glu, transpose_b=True)

        attention = tf.nn.softmax(dot_prod)

        z_plus_e = encoder_glu + input_embeds

        tiled_attention = tf.tile(attention, [1, 1, embed_size])

        c = tf.reduce_sum(tiled_attention*z_plus_e, axis=1)
        decoder_glu = tf.reshape(decoder_glu, [-1, embed_size])

        logits = tf.layers.dense(c+decoder_glu, dict_size)

        out = tf.nn.softmax(logits)

        next_id = tf.argmax(out, axis=1)
        next_embeds = tf.nn.embedding_lookup(embeddings, next_id)
        next_embeds = tf.expand_dims(next_embeds, -1)
        decoder_input = tf.concat([decoder_input[:, embed_size:], next_embeds], axis=1)

        next_ids.append(next_id)
        outs.append(logits)

    outs = tf.stack(outs, axis=1)
    next_ids = tf.stack(next_ids, axis=1)

    if TRAIN:
        global_step = tf.train.get_global_step()
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        train_op = tf.train.GradientDescentOptimizer(1e-2).minimize(loss, global_step)
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                train_op=train_op,
                loss=loss)

    elif EVAL:
        loss = tf.losses.softmax_cross_entropy(label_onehot, outs)
        eval_metric_ops = {'acc': tf.metrics.accuracy(labels, next_ids)}
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=loss,
                eval_metric_ops=eval_metric_ops)

    elif PREDICT:
        estimator_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                predictions={'prediction': next_ids})

    else:
        raise Exception('estiamtor spec is invalid')

    return estimator_spec


def main(_):
    fairseq = tf.estimator.Estimator(model_fn)
    fairseq.train(input_fn, steps=10000)
    fairseq.evaluate(input_fn, steps=10)

    pred = fairseq.predict(input_fn)
    for p in pred:
        print('prediction: {}'.format(p['prediction']))

if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run()